In [1]:
# !pip install requests
# !pip install textblob
# !pip install emoji
# !pip install spacy==3.7.2
# !pip install pydantic==1.9.0
# !python -m spacy download en_core_web_sm

In [2]:
import requests
import json
import ast
import re
import pandas as pd
import string,time
import emoji
import spacy

from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize

In [3]:
# api_key = "8265bd1679663a7ea12ac168da84d2e8"
# url = "https://api.themoviedb.org/3/movie/top_rated"
# all_movies = []

# # Get total number of pages
# params = {
#     "api_key": api_key,
#     "language": "en-US",
#     "page": 1
# }
# response = requests.get(url, params=params)
# data = response.json()
# total_pages = data.get("total_pages", 1)

# # Loop through all pages
# for page in range(1, total_pages + 1):
#     params["page"] = page
#     res = requests.get(url, params=params)
#     if res.status_code == 200:
#         movies = res.json().get("results", [])
#         all_movies.extend(movies)
#         # print(f"Saved page {page}/{total_pages}")
#     else:
#         print(f"Failed to fetch page {page}")

# # Save to CSV
# df = pd.DataFrame(all_movies)
# df.to_csv("top_rated_movies_all_pages.csv", index=False)

# print(f"✅ Saved {len(df)} movies from {total_pages} pages.")

In [4]:
df = pd.read_csv('top_rated_movies_all_pages.csv')

In [5]:
df.tail()

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
9995,False,/hKcO8mX9oqfBKlZ8nibJCPSeqMj.jpg,"[28, 12, 53]",1639,en,Speed 2: Cruise Control,A disgruntled former employee hijacks the Seab...,2.7935,/gnK1ocpwUTj24zAktzomOJsD2bu.jpg,1997-06-13,Speed 2: Cruise Control,False,4.609,1674
9996,False,/wpkcvqArMg5YUAPBq7t15Wq5APt.jpg,"[27, 53]",11442,en,Halloween: Resurrection,Reality programmers at DangerTainment select a...,1.9128,/1mlKwbNzJCGzqe4i0ZEJtUUL290.jpg,2002-07-01,Halloween: Resurrection,False,4.607,1303
9997,False,/c6hFzNJIAGT84E6SfuuqEZNJlqH.jpg,[18],13461,it,Melissa P.,"An adolescent girl, living with her mother and...",5.8144,/3fjMLEVozmApiiDF75dnpSIaYZ3.jpg,2005-11-18,Melissa P.,False,4.600,565
9998,False,/7vR0frpPSBhToKKESNfnPfDTY8y.jpg,"[18, 53]",340674,en,Dark Crimes,"Tadek, a Polish detective, becomes suspicious ...",2.2477,/bEwK1r4pmJ9huEjqrZf73NXJvFy.jpg,2016-10-12,Dark Crimes,False,4.600,451
9999,False,/eZvVoJc3XrAE81bmNhtyTUDti8b.jpg,"[35, 27, 53]",16780,en,The Return of the Texas Chainsaw Massacre,After a group of Texas teenagers leave prom ni...,2.6221,/e1ikqofvLBddrZJ73IjmJWhUYus.jpg,1995-09-22,The Return of the Texas Chainsaw Massacre,False,4.597,694


In [6]:
df.shape

(10000, 14)

In [7]:
# Only select 3 columns to work with

df = df[['original_title','overview','genre_ids']]

df

,original_title,overview,genre_ids
0,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...,"[18, 80]"
1,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","[18, 80]"
2,The Godfather Part II,In the continuing saga of the Corleone crime f...,"[18, 80]"
3,Schindler's List,The true story of how businessman Oskar Schind...,"[18, 36, 10752]"
4,12 Angry Men,The defense and the prosecution have rested an...,[18]
...,...,...,...
9995,Speed 2: Cruise Control,A disgruntled former employee hijacks the Seab...,"[28, 12, 53]"
9996,Halloween: Resurrection,Reality programmers at DangerTainment select a...,"[27, 53]"
9997,Melissa P.,"An adolescent girl, living with her mother and...",[18]
9998,Dark Crimes,"Tadek, a Polish detective, becomes suspicious ...","[18, 53]"


In [8]:
## Fetching movie genres from other api

# Define API URL
url = "https://api.themoviedb.org/3/genre/movie/list"
params = {
    "api_key": "8265bd1679663a7ea12ac168da84d2e8",
    "language": "en-US"
}

# Make the request
response = requests.get(url, params=params)

# Check if successful
if response.status_code == 200:
    data = response.json()
    genres = data.get('genres', [])
else:
    print(f"Failed to fetch data: {response.status_code}")
    genres = []

genre_mapping = {g['id']: g['name'] for g in genres}

genre_mapping

{28: 'Action',
 12: 'Adventure',
 16: 'Animation',
 35: 'Comedy',
 80: 'Crime',
 99: 'Documentary',
 18: 'Drama',
 10751: 'Family',
 14: 'Fantasy',
 36: 'History',
 27: 'Horror',
 10402: 'Music',
 9648: 'Mystery',
 10749: 'Romance',
 878: 'Science Fiction',
 10770: 'TV Movie',
 53: 'Thriller',
 10752: 'War',
 37: 'Western'}

In [9]:
# merge genre with ids

def get_genre_names(genre_ids):
    genre_names = [genre_mapping.get(int(g_id), "Unknown") for g_id in genre_ids]
    return ', '.join(genre_names)

df['genre_ids'] = df['genre_ids'].apply(ast.literal_eval) 
df['genre_names'] = df['genre_ids'].apply(get_genre_names)

df

,original_title,overview,genre_ids,genre_names
0,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...,"[18, 80]","Drama, Crime"
1,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","[18, 80]","Drama, Crime"
2,The Godfather Part II,In the continuing saga of the Corleone crime f...,"[18, 80]","Drama, Crime"
3,Schindler's List,The true story of how businessman Oskar Schind...,"[18, 36, 10752]","Drama, History, War"
4,12 Angry Men,The defense and the prosecution have rested an...,[18],Drama
...,...,...,...,...
9995,Speed 2: Cruise Control,A disgruntled former employee hijacks the Seab...,"[28, 12, 53]","Action, Adventure, Thriller"
9996,Halloween: Resurrection,Reality programmers at DangerTainment select a...,"[27, 53]","Horror, Thriller"
9997,Melissa P.,"An adolescent girl, living with her mother and...",[18],Drama
9998,Dark Crimes,"Tadek, a Polish detective, becomes suspicious ...","[18, 53]","Drama, Thriller"


In [10]:
df['overview'][3].lower()

'the true story of how businessman oskar schindler saved over a thousand jewish lives from the nazis while they worked as slaves in his factory during world war ii.'

## 1. Remove Punctuations

In [11]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [12]:
exclude = string.punctuation

def remove_punc(text):
    return text.translate(str.maketrans('', '', exclude))

df['description_corrected'] = df['overview'].apply(remove_punc)

In [13]:
df['description_corrected'][5]

'A young girl Chihiro becomes trapped in a strange new world of spirits When her parents undergo a mysterious transformation she must call upon the courage she never knew she had to free her family'

## 2. Chat Word Treatment

In [14]:
# Open and read the chat_words.txt file
file_path = 'D:/oishee/jobs/2025/NLP/kaggle_movie_review/datasets/slang.txt'

# Create an empty dictionary
chat_words = {}

# Read and process the file
with open(file_path, 'r', encoding='utf-8') as file:
     for line in file:
        if line.strip():
            parts = re.split(r'\s*[:=–—\-]{1,2}>\s*|\s*[:=–—\-]\s*', line.strip(), maxsplit=1)
            if len(parts) == 2:
                key, value = parts
                chat_words[key.strip().upper()] = value.strip()

In [15]:
print(list(chat_words.items())[:5])  # Show first 5 key-value pairs

[('AFAIK', 'As Far As I Know'), ('AFK', 'Away From Keyboard'), ('ASAP', 'As Soon As Possible'), ('ATK', 'At The Keyboard'), ('ATM', 'At The Moment')]


In [16]:
def chat_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words:
            new_text.append(chat_words[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

df['description_corrected'] = df['description_corrected'].apply(chat_conversion)

In [17]:
df['description_corrected'][1000]

'Over the course of ten days and 435 miles an unbreakable bond is forged between pro adventure racer Michael Light and a scrappy street dog companion dubbed Arthur As the team is pushed to their outer limits of endurance in the race Arthur redefines what victory loyalty and friendship truly mean'

## 3. Lower Casing

In [18]:
df['description_corrected'] = df['description_corrected'].str.lower()

df

,original_title,overview,genre_ids,genre_names,description_corrected
0,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...,"[18, 80]","Drama, Crime",imprisoned in the 1940s for the double murder ...
1,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","[18, 80]","Drama, Crime",spanning the years 1945 to 1955 a chronicle of...
2,The Godfather Part II,In the continuing saga of the Corleone crime f...,"[18, 80]","Drama, Crime",in the continuing saga of the corleone crime f...
3,Schindler's List,The true story of how businessman Oskar Schind...,"[18, 36, 10752]","Drama, History, War",the true story of how businessman oskar schind...
4,12 Angry Men,The defense and the prosecution have rested an...,[18],Drama,the defense and the prosecution have rested an...
...,...,...,...,...,...
9995,Speed 2: Cruise Control,A disgruntled former employee hijacks the Seab...,"[28, 12, 53]","Action, Adventure, Thriller",a disgruntled former employee hijacks the seab...
9996,Halloween: Resurrection,Reality programmers at DangerTainment select a...,"[27, 53]","Horror, Thriller",reality programmers at dangertainment select a...
9997,Melissa P.,"An adolescent girl, living with her mother and...",[18],Drama,an adolescent girl living with her mother and ...
9998,Dark Crimes,"Tadek, a Polish detective, becomes suspicious ...","[18, 53]","Drama, Thriller",tadek a polish detective becomes suspicious of...


## 4. Spelling Correction

In [19]:
# def correct_text(text):
#     try:
#         return (TextBlob(text).correct().astype(str))
#     except:
#         return text

# df['description_corrected'] = df['description_corrected'].apply(correct_text)

In [20]:
df['description_corrected'][5000]

'after telling the story of flints last journey to young jim hawkins billy bones has a heart attack and dies just as jim and his friends are attacked by pirates the gang escapes into the town where they hire out a boat and crew to find the hidden treasure which was revealed by bones before he died on their voyage across the seas they soon find out that not everyone on board can be trusted'

## 5. Removing Stopwords

In [21]:
# import nltk
# nltk.download('stopwords')

In [22]:
print(list(stopwords.words('english')[:5]))  # Show first 5 key-value pairs

['a', 'about', 'above', 'after', 'again']


In [23]:
def remove_stopwords(text):
    new_text = []
    
    for word in text.split():
        if word in stopwords.words('english'):
            new_text.append('')
        else:
            new_text.append(word)
    x = new_text[:]
    new_text.clear()
    return " ".join(x)

df['description_corrected'] = df['description_corrected'].apply(remove_stopwords)

In [24]:
df['description_corrected'][9999]

'  group  texas teenagers leave prom night early  get   car crash   woods  employ  aid   nearby insurance agent  calls   tow truckdriving boyfriend vilmer slaughter unfortunately   vilmer   sadistic killer   mechanical leg  introduces    brother leatherface   rest   twisted family'

## 6. Handling Emojis

In [25]:
df['description_corrected'] = df['description_corrected'].apply(emoji.demojize)

df['description_corrected'][9998]

'tadek  polish detective becomes suspicious   controversial author   incidents described   unpublished novel resemble  inner workings   unsolved murder'

## 7. Tokenization

In [26]:
nlp = spacy.load('en_core_web_sm')

df['description_tokenized'] = df['description_corrected'].apply(nlp)

df['description_tokenized']

0       (imprisoned,   , 1940s,   , double, murder,   ...
1       (spanning,  , years, 1945,  , 1955,  , chronic...
2       (  , continuing, saga,   , corleone, crime, fa...
3       ( , true, story,   , businessman, oskar, schin...
4       ( , defense,   , prosecution,  , rested,   , j...
                              ...                        
9995    ( , disgruntled, former, employee, hijacks,  ,...
9996    (reality, programmers,  , dangertainment, sele...
9997    ( , adolescent, girl, living,   , mother,   , ...
9998    (tadek,  , polish, detective, becomes, suspici...
9999    (  , group,  , texas, teenagers, leave, prom, ...
Name: description_tokenized, Length: 10000, dtype: object

In [27]:
df['description_tokenized'][5000]

 telling  story  flints last journey  young jim hawkins billy bones   heart attack  dies   jim   friends  attacked  pirates  gang escapes   town   hire   boat  crew  find  hidden treasure   revealed  bones   died   voyage across  seas  soon find    everyone  board   trusted

## 8. Stemming

In [28]:
# Initialize the Porter Stemmer
ps = PorterStemmer()

def stem_words(text):
    if not isinstance(text, str):
        return ""
    tokens = word_tokenize(text)
    stems = [ps.stem(word) for word in tokens]
    return " ".join(stems).strip()

In [29]:
df['description_corrected_stemming'] = df['description_corrected'].apply(stem_words)

In [30]:
df['description_corrected_stemming'][5000]

'tell stori flint last journey young jim hawkin billi bone heart attack die jim friend attack pirat gang escap town hire boat crew find hidden treasur reveal bone die voyag across sea soon find everyon board trust'

## Lemmatization

In [31]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [32]:
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    if not isinstance(text, str):
        return ""
    tokens = word_tokenize(text)
    lemmatized = [lemmatizer.lemmatize(word) for word in tokens]
    return " ".join(lemmatized)

In [33]:
df['description_corrected_lemmatize'] = df['description_corrected'].apply(lemmatize_text)

df['description_corrected_lemmatize'][10]

'burgerloving hit man philosophical partner drugaddled gangster moll washedup boxer converge sprawling comedic crime caper adventure unfurl three story ingeniously trip back forth tear eye'

In [35]:
df['description_corrected_lemmatize'][5000]

'telling story flint last journey young jim hawkins billy bone heart attack dy jim friend attacked pirate gang escape town hire boat crew find hidden treasure revealed bone died voyage across sea soon find everyone board trusted'

## Cleaning Corpus

In [36]:
df

,original_title,overview,genre_ids,genre_names,description_corrected,description_tokenized,description_corrected_stemming,description_corrected_lemmatize
0,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...,"[18, 80]","Drama, Crime",imprisoned 1940s double murder wife lo...,"(imprisoned, , 1940s, , double, murder, ...",imprison 1940 doubl murder wife lover upstand ...,imprisoned 1940s double murder wife lover upst...
1,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","[18, 80]","Drama, Crime",spanning years 1945 1955 chronicle fictio...,"(spanning, , years, 1945, , 1955, , chronic...",span year 1945 1955 chronicl fiction italianam...,spanning year 1945 1955 chronicle fictional it...
2,The Godfather Part II,In the continuing saga of the Corleone crime f...,"[18, 80]","Drama, Crime",continuing saga corleone crime family you...,"( , continuing, saga, , corleone, crime, fa...",continu saga corleon crime famili young vito c...,continuing saga corleone crime family young vi...
3,Schindler's List,The true story of how businessman Oskar Schind...,"[18, 36, 10752]","Drama, History, War",true story businessman oskar schindler save...,"( , true, story, , businessman, oskar, schin...",true stori businessman oskar schindler save th...,true story businessman oskar schindler saved t...
4,12 Angry Men,The defense and the prosecution have rested an...,[18],Drama,defense prosecution rested jury filing ...,"( , defense, , prosecution, , rested, , j...",defens prosecut rest juri file juri room decid...,defense prosecution rested jury filing jury ro...
...,...,...,...,...,...,...,...,...
9995,Speed 2: Cruise Control,A disgruntled former employee hijacks the Seab...,"[28, 12, 53]","Action, Adventure, Thriller",disgruntled former employee hijacks seabourn...,"( , disgruntled, former, employee, hijacks, ,...",disgruntl former employe hijack seabourn legen...,disgruntled former employee hijack seabourn le...
9996,Halloween: Resurrection,Reality programmers at DangerTainment select a...,"[27, 53]","Horror, Thriller",reality programmers dangertainment select gr...,"(reality, programmers, , dangertainment, sele...",realiti programm dangertain select group thril...,reality programmer dangertainment select group...
9997,Melissa P.,"An adolescent girl, living with her mother and...",[18],Drama,adolescent girl living mother grandmother...,"( , adolescent, girl, living, , mother, , ...",adolesc girl live mother grandmoth first sexua...,adolescent girl living mother grandmother firs...
9998,Dark Crimes,"Tadek, a Polish detective, becomes suspicious ...","[18, 53]","Drama, Thriller",tadek polish detective becomes suspicious c...,"(tadek, , polish, detective, becomes, suspici...",tadek polish detect becom suspici controversi ...,tadek polish detective becomes suspicious cont...


In [37]:
def cleaned_corpus(doc):
    return [token.text for token in doc if token.text.strip() != '']

df['description_corrected'] = df['description_tokenized'].apply(cleaned_corpus)

In [38]:
df['description_corrected']

0       [imprisoned, 1940s, double, murder, wife, love...
1       [spanning, years, 1945, 1955, chronicle, ficti...
2       [continuing, saga, corleone, crime, family, yo...
3       [true, story, businessman, oskar, schindler, s...
4       [defense, prosecution, rested, jury, filing, j...
                              ...                        
9995    [disgruntled, former, employee, hijacks, seabo...
9996    [reality, programmers, dangertainment, select,...
9997    [adolescent, girl, living, mother, grandmother...
9998    [tadek, polish, detective, becomes, suspicious...
9999    [group, texas, teenagers, leave, prom, night, ...
Name: description_corrected, Length: 10000, dtype: object

#### How many words are there in the entire corpus? 

In [39]:
word_count = df['description_corrected'].apply(lambda x: len(str(x).split())).sum()

word_count

265041

#### Total number of unique words (vocabulary)?

In [40]:
unique_word_count = df['description_corrected'].apply(lambda x: len(str(x).split())).unique().sum()

unique_word_count

4325

## Bag of Words

In [41]:
from sklearn.feature_extraction.text import CountVectorizer

In [42]:
df['description_corpus'] = df['description_corrected'].apply(
    lambda tokens: " ".join(tokens) if isinstance(tokens, list) else ""
)

In [43]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['description_corpus'])

In [44]:
df['description_corpus']

0       imprisoned 1940s double murder wife lover upst...
1       spanning years 1945 1955 chronicle fictional i...
2       continuing saga corleone crime family young vi...
3       true story businessman oskar schindler saved t...
4       defense prosecution rested jury filing jury ro...
                              ...                        
9995    disgruntled former employee hijacks seabourn l...
9996    reality programmers dangertainment select grou...
9997    adolescent girl living mother grandmother firs...
9998    tadek polish detective becomes suspicious cont...
9999    group texas teenagers leave prom night early g...
Name: description_corpus, Length: 10000, dtype: object

In [45]:
# Displaying the Vocabulary and Frequency Counts
print(list(vectorizer.vocabulary_)[:50])

['imprisoned', '1940s', 'double', 'murder', 'wife', 'lover', 'upstanding', 'banker', 'andy', 'dufresne', 'begins', 'new', 'life', 'shawshank', 'prison', 'puts', 'accounting', 'skills', 'work', 'amoral', 'warden', 'long', 'stretch', 'comes', 'admired', 'inmates', 'including', 'older', 'prisoner', 'named', 'red', 'integrity', 'unquenchable', 'sense', 'hope', 'spanning', 'years', '1945', '1955', 'chronicle', 'fictional', 'italianamerican', 'corleone', 'crime', 'family', 'organized', 'patriarch', 'vito', 'barely', 'survives']


In [46]:
print("Feature Names:", vectorizer.get_feature_names_out())

Feature Names: ['00' '006' '007' ... 'éric' 'öztürk' 'ʻohana']


In [47]:
bow_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

bow_df

,00,006,007,0s,10,100,1000,10000,100000,10000000,...,ángel,ángela,æon,édouard,émigré,émile,émilie,éric,öztürk,ʻohana
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
import numpy as np
# Analyzing Word Frequencies
word_counts = np.sum(X.toarray(), axis=0)

array([2, 1, 5, ..., 2, 2, 2], dtype=int64)

In [52]:
word_freq = dict(zip(vectorizer.get_feature_names_out(), word_counts))
print("Word Frequencies: ")
word_freq

Word Frequencies: 


{'00': 2,
 '006': 1,
 '007': 5,
 '0s': 1,
 '10': 32,
 '100': 13,
 '1000': 6,
 '10000': 5,
 '100000': 2,
 '10000000': 1,
 '1000foot': 1,
 '1000page': 1,
 '1000th': 1,
 '1001': 1,
 '100acrewood': 1,
 '100foot': 1,
 '100th': 3,
 '101': 2,
 '10191': 1,
 '101yearold': 1,
 '1021': 1,
 '103': 1,
 '108yearold': 1,
 '10day': 1,
 '10million': 1,
 '10round': 1,
 '10s': 1,
 '10th': 4,
 '10year': 3,
 '10yearold': 17,
 '11': 8,
 '111': 3,
 '1114pm': 1,
 '1123': 1,
 '1138': 1,
 '117': 4,
 '118': 2,
 '118th': 1,
 '11th': 3,
 '11year': 2,
 '11yearold': 16,
 '11yearolds': 1,
 '12': 26,
 '120': 2,
 '1200': 3,
 '1215': 1,
 '1250': 1,
 '125000': 1,
 '12foot': 1,
 '12hour': 2,
 '12person': 1,
 '12step': 1,
 '12th': 5,
 '12thcentury': 1,
 '12year': 1,
 '12yearold': 23,
 '13': 23,
 '130': 1,
 '130000': 1,
 '1347': 1,
 '13th': 2,
 '13week': 1,
 '13yearold': 14,
 '14': 14,
 '140': 1,
 '1408': 1,
 '1415': 1,
 '142': 1,
 '1429': 1,
 '1492': 2,
 '14th': 4,
 '14thcentury': 1,
 '14year': 1,
 '14yearold': 11,
 '15': 

## Bag of Bi-gram And Bag of Tri-gram

In [61]:
cv = CountVectorizer(ngram_range=(2,3))

X = cv.fit_transform(df['description_corpus'])

In [62]:
# vocab
print(list(cv.vocabulary_.items())[:20])

[('imprisoned 1940s', 196759), ('1940s double', 1224), ('double murder', 109757), ('murder wife', 264126), ('wife lover', 432946), ('lover upstanding', 238021), ('upstanding banker', 417388), ('banker andy', 32073), ('andy dufresne', 17717), ('dufresne begins', 113107), ('begins new', 37848), ('new life', 272718), ('life shawshank', 228340), ('shawshank prison', 351895), ('prison puts', 308026), ('puts accounting', 313483), ('accounting skills', 5697), ('skills work', 358105), ('work amoral', 438021), ('amoral warden', 16872)]


In [65]:
# vocab
print(list(cv.vocabulary_.items())[-20:])

[('aid nearby insurance', 11166), ('nearby insurance agent', 270066), ('insurance agent calls', 200808), ('agent calls tow', 10013), ('calls tow truckdriving', 55019), ('tow truckdriving boyfriend', 400027), ('truckdriving boyfriend vilmer', 406326), ('boyfriend vilmer slaughter', 47687), ('vilmer slaughter unfortunately', 422375), ('slaughter unfortunately vilmer', 358415), ('unfortunately vilmer sadistic', 414299), ('vilmer sadistic killer', 422373), ('sadistic killer mechanical', 337301), ('killer mechanical leg', 215169), ('mechanical leg introduces', 250165), ('leg introduces brother', 224986), ('introduces brother leatherface', 202147), ('brother leatherface rest', 50731), ('leatherface rest twisted', 223642), ('rest twisted family', 327036)]


In [66]:
len(cv.vocabulary_.items())

447501

## TF-IDF

In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [70]:
tfidf = TfidfVectorizer()
tfidf.fit_transform(df['description_corpus']).toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [71]:
print(tfidf.idf_) # idf is fixed for each doc

[9.11182808 9.51729319 8.4186809  ... 9.51729319 9.51729319 9.11182808]


In [72]:
print(tfidf.get_feature_names_out())

['00' '006' '007' ... 'éric' 'öztürk' 'ʻohana']


In [73]:
# vocab
print(list(tfidf.vocabulary_.items())[:20])

[('imprisoned', 13614), ('1940s', 201), ('double', 8128), ('murder', 18255), ('wife', 29789), ('lover', 16262), ('upstanding', 28778), ('banker', 2533), ('andy', 1486), ('dufresne', 8383), ('begins', 2861), ('new', 18671), ('life', 15854), ('shawshank', 24502), ('prison', 21257), ('puts', 21643), ('accounting', 718), ('skills', 24969), ('work', 30025), ('amoral', 1408)]
